In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymongo
# import iso639
import time
from PIL import Image
from pprint import pprint
from wordcloud import WordCloud

### Conexão com o MongoDB

In [2]:
client = pymongo.MongoClient()
db = client['8M2020_samples']  # client.tweets8march
collection = db.tweets_pt  # db.tweets

In [3]:
def query(collection, pipeline):
    time_start = time.time()
    query = list(collection.aggregate(pipeline, allowDiskUse=True))
    time_end = time.time()
    print(
        f'Tempo de execução: {time.strftime("%H:%M:%S", time.gmtime(time_end - time_start))}')
    return query

### Número de documentos (tweets)

In [4]:
num_docs = collection.count_documents({})
num_docs

274636

### Número de perfis presentes na base

In [5]:
pipeline1 = [
    {
        '$group': {
            '_id': '$user.screen_name', 
            'count': {
                '$sum': 1
            }
        }
    }, {
        '$count': 'qtde_users'
    }
]

In [6]:
qtde_users = query(collection=collection, pipeline=pipeline1)
num_users = dict(qtde_users[0])['qtde_users']
print(f'Número de usuários: {num_users}')

Tempo de execução: 00:00:00
Número de usuários: 206600


### Aggregation usada para os próximos números

In [7]:
pipeline2 = [
    {
        '$project': {
            'tweet_id': '$id_str', 
            'date': {
                '$dateFromString': {
                    'dateString': '$created_at'
                }
            }, 
            'tweet_text': {
                '$ifNull': [
                    '$retweeted_status.full_text', '$full_text'
                ]
            }, 
            'num_likes': '$favorite_count', 
            'num_retweets': '$retweet_count', 
            'quoted_status_id': {
                '$ifNull': [
                    '$quoted_status_id_str', None
                ]
            }, 
            'retweeted_status_id': {
                '$ifNull': [
                    '$retweeted_status.id_str', None
                ]
            }, 
            'reply_to_user': {
                '$ifNull': [
                    '$in_reply_to_user_id_str', None
                ]
            }, 
            'reply_to_status': {
                '$ifNull': [
                    '$in_reply_to_status_id_str', None
                ]
            }, 
            'user_id': '$user.id_str', 
            'screen_name': '$user.screen_name', 
            'followers': '$user.followers_count', 
            'following': '$user.friends_count'
        }
    }, {
        '$project': {
            '_id': 0, 
            'tweet_id': 1, 
            'date': {
                '$dateToString': {
                    'format': '%d/%m/%Y %H:%M:%S', 
                    'date': '$date'
                }
            }, 
            'tweet_text': 1, 
            'num_likes': 1, 
            'num_retweets': 1, 
            'quoted_status_id': 1, 
            'retweeted_status_id': 1, 
            'reply_to_user': 1, 
            'reply_to_status': 1, 
            'user_id': 1, 
            'screen_name': 1, 
            'followers': 1, 
            'following': 1
        }
    }
]

In [8]:
docs = query(collection=collection, pipeline=pipeline2)

Tempo de execução: 00:00:05


In [9]:
docs[0]

{'tweet_id': '1236411302895124487',
 'tweet_text': 'Sol com cara O projeto Inclusive, Praia! está de volta no próximo domingo (8) com uma programação especial em referência ao Dia Internacional da Mulher. Vai ter jogos, brincadeiras, diversão e show com Isabelle Passinho. 🏝 Não perca! &gt; https://t.co/fYin5VeZwx \n#GovernoDeTodosNós https://t.co/YIGahguJN0',
 'num_likes': 8,
 'num_retweets': 2,
 'quoted_status_id': None,
 'retweeted_status_id': None,
 'reply_to_user': None,
 'reply_to_status': None,
 'user_id': '53085255',
 'screen_name': 'GovernoMA',
 'followers': 175381,
 'following': 166,
 'date': '07/03/2020 22:00:00'}

In [10]:
docs_df = pd.DataFrame(docs)
docs_df = docs_df[['tweet_id', 'date', 'tweet_text', 'num_likes', 'num_retweets',
                   'retweeted_status_id', 'quoted_status_id', 'reply_to_user', 'reply_to_status',
                   'user_id', 'screen_name', 'followers', 'following']]

In [11]:
docs_df.head()

,tweet_id,date,tweet_text,num_likes,num_retweets,retweeted_status_id,quoted_status_id,reply_to_user,reply_to_status,user_id,screen_name,followers,following
0,1236411302895124487,07/03/2020 22:00:00,"Sol com cara O projeto Inclusive, Praia! está ...",8,2,None,None,None,None,53085255,GovernoMA,175381,166
1,1236411378879037442,07/03/2020 22:00:18,Hoje é dia de transmissão especial na Fla TV!!...,0,36,1236371617732874242,None,None,None,79617245,audreysteen,38,342
2,1236411364643659776,07/03/2020 22:00:15,"A presidenta nacional do PT, deputada @gleisi ...",0,184,1236375681245876225,None,None,None,139207105,ProfValterCosta,1758,2800
3,1236411353444868096,07/03/2020 22:00:12,Cartaz de minha enteada querida(linda e talent...,0,3,1236407917554929666,None,None,None,1093274352060121088,purpurediamond,28,90
4,1236411366086508544,07/03/2020 22:00:15,Dia Internacional da Mulher: o que os homens p...,2,0,None,None,None,None,126318634,professoraAlice,26485,170


### Número de tweets únicos

In [12]:
num_texts = len(docs_df['tweet_text'].value_counts())
num_texts

48942

In [15]:
text_df = pd.DataFrame(docs_df['tweet_text'].value_counts(
).rename_axis('unique_values').reset_index(name='counts'))

pd.set_option('display.max_colwidth', None)
text_df.head(10)

,unique_values,counts
0,E o motorista do 630 que todo ano na semana do dia internacional da mulher enfeita o ônibus todo e dar uma lembrancinha para todas as mulheres que entram no ônibus 😍😍❤️❤️😭 https://t.co/SNSfrNM6L6,18798
1,"""8 de março de 1857, morreram aproximadamente 130 mulheres carbonizadas quando foram trancadas na fábrica de tecelagem, em Nova York, onde trabalhavam, por estarem em greve. Em homenagem a essas mulheres, em 1910, declarou-se o dia 8 de março como o ""Dia Internacional da Mulher"". https://t.co/VyABZO6WrU",17188
2,"dia internacional da mulher, não pode faltar https://t.co/MRYwQyhx14",13179
3,Somos incríveis. #diadasmulheres #DiaDaMulher https://t.co/XnrdVS4WH0,11289
4,no dia internacional da mulher vamos relembrar a importância desse vídeo icônico https://t.co/P31255LPOJ,7779
5,feliz #diadasmulheres com o tik tok mais feminista possível https://t.co/UPn9fUA9Ue,6169
6,"Bom dia...Hoje é o Dia Internacional da Mulher. E esses números mostram a importância dessa data. Não é sobre flores, e presentes. É sobre respeitarem nossos direitos. A luta segue. https://t.co/X3qE1DsjTr",3830
7,"Lugar de mulher é onde ela quiser, inclusive na Justiça e Segurança Pública. Homenagem da Força Nacional/MJSP no Dia Internacional das Mulheres. https://t.co/uApHnqD6AI",3574
8,Só isso que eu vou falar. #diadasmulheres https://t.co/lTztsSrrP2,3486
9,Hoje é Dia Internacional da Mulher e eu só lembro do dia que o Mc Donalds colocou todos os machos de folga pra que as mulheres trabalhassem e ainda trataram isso como ato “feminista”. https://t.co/BlyXDy4lQ9,3260


In [16]:
text_df['unique_values'][0]

'E o motorista do 630 que todo ano na semana do dia internacional da mulher enfeita o ônibus todo e dar uma lembrancinha para todas as mulheres que entram no ônibus 😍😍❤️❤️😭 https://t.co/SNSfrNM6L6'

### Número de tweets que não se repetem (sem RT)

In [17]:
index = text_df['counts']==1

In [18]:
num_one_text = len(text_df[index])
num_one_text

38334

### Número de tweets que se repetem

In [19]:
num_repeat_text = num_texts - num_one_text
num_repeat_text

10608

In [20]:
text_df.head(10)

,unique_values,counts
0,E o motorista do 630 que todo ano na semana do dia internacional da mulher enfeita o ônibus todo e dar uma lembrancinha para todas as mulheres que entram no ônibus 😍😍❤️❤️😭 https://t.co/SNSfrNM6L6,18798
1,"""8 de março de 1857, morreram aproximadamente 130 mulheres carbonizadas quando foram trancadas na fábrica de tecelagem, em Nova York, onde trabalhavam, por estarem em greve. Em homenagem a essas mulheres, em 1910, declarou-se o dia 8 de março como o ""Dia Internacional da Mulher"". https://t.co/VyABZO6WrU",17188
2,"dia internacional da mulher, não pode faltar https://t.co/MRYwQyhx14",13179
3,Somos incríveis. #diadasmulheres #DiaDaMulher https://t.co/XnrdVS4WH0,11289
4,no dia internacional da mulher vamos relembrar a importância desse vídeo icônico https://t.co/P31255LPOJ,7779
5,feliz #diadasmulheres com o tik tok mais feminista possível https://t.co/UPn9fUA9Ue,6169
6,"Bom dia...Hoje é o Dia Internacional da Mulher. E esses números mostram a importância dessa data. Não é sobre flores, e presentes. É sobre respeitarem nossos direitos. A luta segue. https://t.co/X3qE1DsjTr",3830
7,"Lugar de mulher é onde ela quiser, inclusive na Justiça e Segurança Pública. Homenagem da Força Nacional/MJSP no Dia Internacional das Mulheres. https://t.co/uApHnqD6AI",3574
8,Só isso que eu vou falar. #diadasmulheres https://t.co/lTztsSrrP2,3486
9,Hoje é Dia Internacional da Mulher e eu só lembro do dia que o Mc Donalds colocou todos os machos de folga pra que as mulheres trabalhassem e ainda trataram isso como ato “feminista”. https://t.co/BlyXDy4lQ9,3260


### Relatório

In [21]:
from prettytable import PrettyTable
results = PrettyTable()

In [22]:
results.field_names = ['Métrica', 'Quantidade']
results.add_row(['Documentos', num_docs])
results.add_row(['Perfis', num_users])
results.add_row(['Tweets únicos', num_texts])
results.add_row(['Tweets que se repetem', num_repeat_text])
results.add_row(['Tweets que não se repetem', num_one_text])

In [23]:
print(results)

+---------------------------+------------+
|          Métrica          | Quantidade |
+---------------------------+------------+
|         Documentos        |   274636   |
|           Perfis          |   206600   |
|       Tweets únicos       |   48942    |
|   Tweets que se repetem   |   10608    |
| Tweets que não se repetem |   38334    |
+---------------------------+------------+


### geolocalização

In [24]:
pipeline3 = [
    {
        '$match': {
            'geo': {
                '$ne': None
            }
        }
    }, {
        '$count': 'count'
    }
]

In [25]:
geo_enabled = query(collection=collection, pipeline=pipeline3)

Tempo de execução: 00:00:00


In [26]:
geo_enabled

[{'count': 712}]

### user geo enabled

In [27]:
pipeline4 = [
    {
        '$project': {
            'user_geo_enabled': '$user.geo_enabled', 
            'user_location': '$user.location'
        }
    }, {
        '$group': {
            '_id': '$user_geo_enabled', 
            'count': {
                '$sum': 1
            }
        }
    }
]

In [28]:
user_geo_enabled = query(collection=collection, pipeline=pipeline4)

Tempo de execução: 00:00:00


In [29]:
user_geo_enabled

[{'_id': False, 'count': 159596}, {'_id': True, 'count': 115040}]

### user location

In [30]:
pipeline5 = [
    {
        '$project': {
            'user_geo_enabled': '$user.geo_enabled', 
            'user_location': '$user.location'
        }
    }, {
        '$group': {
            '_id': '$user_location', 
            'count': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'count': -1
        }
    }
]

In [31]:
user_location = query(collection=collection, pipeline=pipeline5)

Tempo de execução: 00:00:00


In [19]:
user_location

[{'_id': '', 'count': 80069},
 {'_id': 'Rio de Janeiro, Brasil', 'count': 10777},
 {'_id': 'Brasil', 'count': 8865},
 {'_id': 'São Paulo, Brasil', 'count': 6843},
 {'_id': 'Belo Horizonte, Brasil', 'count': 2063},
 {'_id': 'Brasília, Brasil', 'count': 1982},
 {'_id': 'Rio de Janeiro', 'count': 1897},
 {'_id': 'Brazil', 'count': 1402},
 {'_id': 'São Paulo', 'count': 1336},
 {'_id': 'Minas Gerais, Brasil', 'count': 1167},
 {'_id': 'Curitiba, Brasil', 'count': 1123},
 {'_id': 'Recife, Brasil', 'count': 1114},
 {'_id': 'Porto Alegre, Brasil', 'count': 1035},
 {'_id': 'Fortaleza, Brasil', 'count': 1028},
 {'_id': 'Rio Grande do Sul, Brasil', 'count': 1013},
 {'_id': 'Manaus, Brasil', 'count': 921},
 {'_id': '021', 'count': 839},
 {'_id': 'Goiânia, Brasil', 'count': 795},
 {'_id': 'Salvador, Brasil', 'count': 755},
 {'_id': 'Santa Catarina, Brasil', 'count': 705},
 {'_id': 'Portugal', 'count': 701},
 {'_id': 'Espírito Santo, Brasil', 'count': 647},
 {'_id': 'Sao Paulo, Brazil', 'count': 621}

### Códigos bcp47 (usado no atributo __lang__)

In [ ]:
# iso639.languages.get(alpha2 = 'en').name
bcp_lang = pd.read_csv('bcp47.csv')
bcp_lang.head(15)

### Idiomas mais presentes na base

In [ ]:
pipeline = [
    {
        '$group': {
            '_id': '$lang',
            'count': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'count': -1
        }
    }
]

In [ ]:
time_start = time.time()
languages = list(collection.aggregate(pipeline, allowDiskUse=True))
time_end = time.time()

print(
    f'Tempo de execução: {time.strftime("%H:%M:%S", time.gmtime(time_end - time_start))}')

In [ ]:
languages_df = pd.DataFrame(languages)
languages_df.rename(columns={"_id": "language"}, inplace=True)
languages_df.head(15)

In [ ]:
def get_language(lang):
    language = bcp_lang[bcp_lang['lang'] == lang]['language'].tolist()[0]
    # language = iso639.languages.get(alpha2 = lang).name
    return (language)

In [ ]:
get_language(lang='eu')

In [ ]:
languages_df['language'] = languages_df['language'].apply(get_language)

In [ ]:
languages_df.head(15)

In [ ]:
palavras = ','.join(list(languages_df['language'].values))

In [ ]:
# Generate a word cloud image
wordcloud = WordCloud().generate(palavras)

In [ ]:
# Display the generated image: the matplotlib way:
plt.figure(figsize=(15, 15))
plt.axis("off")
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

# Hashtags mais usadas

In [ ]:
pipeline1 = [
    {
        '$project': {
            '_id': 0,
            'id': 1,
            'retweeted_status': {
                '$ifNull': [
                    '$entities.hashtags.text', '$retweeted_status.extended_tweet.entities.hashtags.text'
                ]
            },
            'lang': 1
        }
    }, {
        '$project': {
            'lang': 1,
            'hashtags': '$retweeted_status'
        }
    }, {
        '$unwind': {
            'path': '$hashtags'
        }
    }, {
        '$group': {
            '_id': '$hashtags',
            'count': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'count': -1
        }
    }
]

In [ ]:
time_start = time.time()
document1 = list(collection.aggregate(pipeline=pipeline1))
time_end = time.time()

print(
    f'Tempo de execução: {time.strftime("%H:%M:%S", time.gmtime(time_end - time_start))}')

In [ ]:
df1 = pd.DataFrame(document1)
df1.rename(columns={"_id": "hashtag"}, inplace=True)
df1.head(15)

In [ ]:
palavras = ','.join(list(df1['hashtag'].values))

# Generate a word cloud image
wordcloud = WordCloud(
    max_font_size=40, font_path='/usr/share/fonts/truetype/freefont/FreeSerif.ttf').generate(palavras)

# Display the generated image: the matplotlib way:
plt.figure(figsize=(15, 15))
plt.axis("off")
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

# Função para obter hashtags mais usadas por idioma

In [ ]:
def get_hashtag_by_lang(lang):
    pipeline = [
        {
            '$match': {
                'lang': lang
            }
        }, {
            '$project': {
                '_id': 0,
                'id': 1,
                'retweeted_status': {
                    '$ifNull': [
                        '$entities.hashtags.text', '$retweeted_status.extended_tweet.entities.hashtags.text'
                    ]
                },
                'lang': 1
            }
        }, {
            '$project': {
                'lang': 1,
                'hashtags': '$retweeted_status'
            }
        }, {
            '$unwind': {
                'path': '$hashtags'
            }
        }, {
            '$group': {
                '_id': '$hashtags',
                'count': {
                    '$sum': 1
                }
            }
        }, {
            '$sort': {
                'count': -1
            }
        }
    ]

    time_start = time.time()
    document = list(collection.aggregate(pipeline=pipeline))
    time_end = time.time()
    print(
        f'Tempo de execução: {time.strftime("%H:%M:%S", time.gmtime(time_end - time_start))}')
    return(document)

# Hashtags mais usadas - tweets em português

In [ ]:
hashtags_pt = get_hashtag_by_lang(lang='pt')

In [ ]:
df_pt = pd.DataFrame(hashtags_pt)
df_pt.rename(columns={"_id": "hashtag"}, inplace=True)
df_pt.head(15)

In [ ]:
palavras = ','.join(list(df_pt['hashtag'].values))

# Generate a word cloud image
wordcloud = WordCloud(
    max_font_size=40, font_path='/usr/share/fonts/truetype/freefont/FreeSerif.ttf').generate(palavras)

# Display the generated image: the matplotlib way:
plt.figure(figsize=(15, 15))
plt.axis("off")
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

# Hashtags mais usadas - tweets em espanhol

In [ ]:
hashtags_es = get_hashtag_by_lang(lang='es')

In [ ]:
df_es = pd.DataFrame(hashtags_es)
df_es.rename(columns={"_id": "hashtag"}, inplace=True)
df_es.head(15)

In [ ]:
palavras_es = ','.join(list(df_es['hashtag'].values))

# Generate a word cloud image
wordcloud = WordCloud(
    max_font_size=40, font_path='/usr/share/fonts/truetype/freefont/FreeSerif.ttf').generate(palavras_es)

# Display the generated image: the matplotlib way:
plt.figure(figsize=(15, 15))
plt.axis("off")
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

# Hashtags mais usadas - tweets em inglês

In [ ]:
hashtags_en = get_hashtag_by_lang(lang='en')

In [ ]:
df_en = pd.DataFrame(hashtags_en)
df_en.rename(columns={"_id": "hashtag"}, inplace=True)
df_en.head(15)

In [ ]:
palavras_en = ','.join(list(df_en['hashtag'].values))

# Generate a word cloud image
wordcloud = WordCloud(
    max_font_size=40, font_path='/usr/share/fonts/truetype/freefont/FreeSerif.ttf').generate(palavras_en)

# Display the generated image: the matplotlib way:
plt.figure(figsize=(15, 15))
plt.axis("off")
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()